# Update a Collection's metadata

The script in this notebook updates the metadata for a private Collection (revision or unpublished).

In order to edit Collections with this script, you must have a Curation API key (obtained from upper-righthand dropdown in the Data Portal UI after logging in).

#### <font color='#bc00b0'>Please fill in the required values:</font>

<font color='#bc00b0'>(Required) Provide the path to your api key file</font>

In [ ]:
api_key_file_path <- "path/to/api-key.txt"

<font color='#bc00b0'>(Required) Enter the id of the Collection that you want to edit</font>

_The Collection id can be found by looking at the url path in the address bar 
when viewing your Collection in the UI of the Data Portal website:_ `collections/{collection_id}`.

In [ ]:
collection_id <- "01234567-89ab-cdef-0123-456789abcdef"

<font color='#bc00b0'>(Required) Provide the updated Collection form metadata for your existing Collection. **Do not include attributes that you do not wish to change**. If a non-empty list of links is passed in, the existing links on the Collection will all be removed and replaced with the list of links that you provide here. If a DOI link is included but the DOI cannot be found on Crossref or is invalid, the entire request will be rejected.</font>

In [ ]:
collection_form_metadata <- list(
  name = "Updated name",
  links = list(
      list(
          link_name = "updated sample protocol link",
          link_url = "https://www.updated_sample.link",
          link_type = "PROTOCOL"
      ),
      list(
          link_name = "updated sample lab website",
          link_url = "https://www.lab.website.updated",
          link_type = "LAB_WEBSITE"
      )
  )
)

### Import dependencies

In [ ]:
library("readr")
library("httr")
library("stringr")
library("rjson")

### Specify domain (and API url)

In [ ]:
domain_name <- "cellxgene.cziscience.com"
site_url <- str_interp("https://${domain_name}")
api_url_base <- str_interp("https://api.${domain_name}")

### Use API key to obtain a temporary access token

In [ ]:
api_key <- read_file(api_key_file_path)
access_token_path <- "/curation/v1/auth/token"
access_token_url <- str_interp("${api_url_base}${access_token_path}")
res <- POST(url=access_token_url, add_headers(`x-api-key`=api_key))
stop_for_status(res)
access_token <- content(res)$access_token

##### (optional, debug) verify status code of response

In [ ]:
print(res$status_code)

### Formulate request and update a Collection

In [ ]:
collection_path <- str_interp("/curation/v1/collections/${collection_id}")
url <- str_interp("${api_url_base}${collection_path}")
bearer_token <- str_interp("Bearer ${access_token}")
res <- PATCH(
    url=url, body=toJSON(collection_form_metadata),
    add_headers(`Authorization`=bearer_token, `Content-Type`="application/json")
)
stop_for_status(res)
res_content <- content(res)  # response data

cat("Updated the Collection at url:\n")
cat(str_interp("${site_url}/collections/${collection_id}"))
